In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pylab
import math
import pandas as pd
from io import StringIO
import random
import scipy
import operator

In [2]:
G_APMS = nx.read_edgelist('yeast_AP-MS.txt')
G_Y2H = nx.read_edgelist('yeast_Y2H.txt')
G_LIT = nx.read_edgelist('yeast_LIT.txt')

G_LIT_Reguly = nx.Graph()
data_lit_reguly = np.genfromtxt("yeast_LIT_Reguly.txt",dtype='unicode',delimiter = '\t')
yeast_LIT_Reguly_raw = [data_lit_reguly[1:,0],data_lit_reguly[1:,1]]

i=1
while i<len(data_lit_reguly[1:,0]):
    G_LIT_Reguly.add_edge(data_lit_reguly[i,0],data_lit_reguly[i,1])
    i+=1

In [3]:
G_ESSENTIAL = nx.Graph()
data_G_ESSENTIAL = np.genfromtxt("Essential_ORFs_paperHe.txt",dtype='unicode',delimiter = '\t')
yeast_G_ESSENTIAL_raw = [data_G_ESSENTIAL[2:,1]]

i = 0
l = len(yeast_G_ESSENTIAL_raw[0][:])

while i < l:
    proteina = yeast_G_ESSENTIAL_raw[0][i]
    yeast_G_ESSENTIAL_raw[0][i] = proteina.rstrip(" ")
    i+=1

Agregamos la esencialidad a los nodos

In [4]:
def agrega_esencialidad(red,lista_esencial):
    
    j = 0
    l = len(red)
    while j < l:
        nombre = list(red)[j]
        red.node[nombre]['esencialidad']=0
        j += 1
    
    for i in lista_esencial[0]:
        if (i in red):
            red.node[i]['esencialidad']=1
    return

In [5]:
agrega_esencialidad(G_APMS,yeast_G_ESSENTIAL_raw)
agrega_esencialidad(G_Y2H,yeast_G_ESSENTIAL_raw)
agrega_esencialidad(G_LIT,yeast_G_ESSENTIAL_raw)
agrega_esencialidad(G_LIT_Reguly,yeast_G_ESSENTIAL_raw)

In [6]:
def analisis_nodos_no_adyacentes(red,vecinos):
    A = nx.adjacency_matrix(red)
    A2 = A * A
    A = A.toarray()
    A2 = A2.toarray()
    contador_tipo_indiferente = 0
    contador_mismo_tipo = 0
    l = nx.number_of_nodes(red)
    i = 0
    while i < l:
        j = i + 1
        while j < l:
            enlace_ij = A[i][j]
            if enlace_ij == 0:    #no son adyacentes
                caminos_ij = A2[i][j]   #cantidad de vecinos en comun
                if caminos_ij >= vecinos:
                    contador_tipo_indiferente += 1
                    nombre_i = list(red)[i]
                    nombre_j = list(red)[j]
                    esencialidad_nodo_i = red.node[nombre_i]['esencialidad']
                    esencialidad_nodo_j = red.node[nombre_j]['esencialidad']   
                    if esencialidad_nodo_i==esencialidad_nodo_j:
                        contador_mismo_tipo +=1
            j += 1
        i += 1
    return (contador_tipo_indiferente,contador_mismo_tipo)
    

In [7]:
pares_no_adyacentes_APMS = analisis_nodos_no_adyacentes(G_APMS,3)
pares_no_adyacentes_Y2H = analisis_nodos_no_adyacentes(G_Y2H,1)
pares_no_adyacentes_LIT = analisis_nodos_no_adyacentes(G_LIT,3)
pares_no_adyacentes_LIT_Reguly = analisis_nodos_no_adyacentes(G_LIT_Reguly,3)

Calculamos la probabilidad Pe de que dos nodos no adyacentes cada uno con grado k1 y k2 tengan n vecinos

In [8]:
def pares_esperados(red,vecinos,alfa,beta,da,db):
    A = nx.adjacency_matrix(red)
    A2 = A * A
    A = A.toarray()
    A2 = A2.toarray()
    proba_acum = 0.0
    proba_acum_error = 0.0
    l = nx.number_of_nodes(red)
    i = 0
    while i < l:
        j = i + 1
        while j < l:
            enlace_ij = A[i][j]
            if enlace_ij == 0:          # no son adyacentes
                caminos_ij = A2[i][j]   #cantidad de vecinos en comun
                if caminos_ij >= vecinos:
                    ki =  list(red.degree)[i][1]
                    kj =  list(red.degree)[j][1]
                    Pei = 1-(1-beta)*(1-alfa)**ki
                    Pej = 1-(1-beta)*(1-alfa)**kj
                    pee = Pei*Pej
                    pnene = (1-Pei)*(1-Pej)
                    proba_acum+= (pee + pnene)
                    proba_acum_error+= 2*Pei*Pej*(ki*da+db)*(kj*da+db) 
            j += 1
        i += 1
    return proba_acum, proba_acum_error

In [9]:
numero_esperado_APMS = pares_esperados(G_APMS,3,0.05,0.2,0.01,0.1)

In [10]:
numero_esperado_LIT_Reguly = pares_esperados(G_LIT_Reguly,3,0.043,0.075,0.004,0.022)

In [11]:
numero_esperado_Y2H = pares_esperados(G_Y2H,1,0.018,0.17,0.013,0.074)

In [12]:
numero_esperado_LIT = pares_esperados(G_LIT,3,0.088,0.23,0.011,0.064)

In [20]:
data = pd.DataFrame({" ":["AP/MS",'Y2H','LIT','LIT-Reguly'],
                     "Total number of pairs":[pares_no_adyacentes_APMS[0],pares_no_adyacentes_Y2H[0],pares_no_adyacentes_LIT[0],pares_no_adyacentes_LIT_Reguly[0]],
                     "Total number of pairs of the same type":[pares_no_adyacentes_APMS[1],pares_no_adyacentes_Y2H[1],pares_no_adyacentes_LIT[1],pares_no_adyacentes_LIT_Reguly[1]],
                     "Expected number of pairs of the same type":[round(numero_esperado_APMS[0],2),numero_esperado_Y2H[0],numero_esperado_LIT[0],numero_esperado_LIT_Reguly[0]],
                     })
data
#round(,2)

,,Expected number of pairs of the same type,Total number of pairs,Total number of pairs of the same type
0,AP/MS,7736.150000,11613,5924
1,Y2H,14164.446468,23073,15019
2,LIT,402.344152,730,393
3,LIT-Reguly,5608.354950,10772,6158
